# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-03 01:02:45] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-03 01:02:45] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-03 01:02:45] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-03 01:02:48] INFO server_args.py:1614: Attention backend not specified. Use fa3 backend by default.


[2026-01-03 01:02:48] INFO server_args.py:2489: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=120 avail_mem=74.64 GB):   5%|▌         | 1/20 [00:00<00:03,  5.02it/s]

Capturing batches (bs=80 avail_mem=74.61 GB):  20%|██        | 4/20 [00:00<00:01, 13.53it/s]

Capturing batches (bs=40 avail_mem=74.59 GB):  50%|█████     | 10/20 [00:00<00:00, 20.42it/s]

Capturing batches (bs=8 avail_mem=74.57 GB):  80%|████████  | 16/20 [00:00<00:00, 20.40it/s] 

Capturing batches (bs=1 avail_mem=74.55 GB): 100%|██████████| 20/20 [00:00<00:00, 20.08it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Linus. I'm a 12-year-old boy from Sweden. I've been teaching myself to play the piano for the last 14 years. I'm an improviser. I never write the same piece, but I always feel like it's unique. I think my style is cool and it makes me feel good. I'm not good at reading music. I can only read music with the help of a teacher. My school teacher is very good at music. I'm really proud of that. I'm so glad to be a musician. I'm also really interested in the history of music and I want to learn more
Prompt: The president of the United States is
Generated text:  retiring and has 100 employees to distribute among 5 different departments. If each employee is to be allocated to one of these departments, in how many ways can the president's 100 employees be distributed among the 5 departments? To determine the number of ways to distribute 100 employees among 5 different departments, we can use the concept of permutations. Each employee has 5 choices of 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm passionate about [job title] and [job title]. I'm always looking for ways to [job title] and I'm always eager to learn new things. I'm always looking for ways to [job title] and I'm always eager to learn new things. I'm always looking for ways to [job title] and I'm always eager to learn new things. I'm always

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic Eiffel Tower, vibrant nightlife, and rich cultural heritage. It is also a major transportation hub and a major tourist destination. Paris is home to many famous landmarks such as Notre-Dame Cathedral, the Louvre Museum, and the Palace of Versailles. The city is also known for its diverse cuisine and its role in the French Revolution and the French Revolution. Paris is a city of contrasts, with its rich history and modernity. It is a city that has been a major influence on French culture and politics for centuries. The city is also home to many important museums, including the Musée d'

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn and adapt to human behavior and preferences. This could lead to more sophisticated and personalized AI systems that can better understand and respond to human needs.

2. Enhanced machine learning capabilities: AI is likely to become more capable of learning and adapting to new situations, allowing machines to perform tasks that were previously impossible. This could lead to more efficient and effective AI systems that can handle a wider range of tasks.

3. Greater emphasis on ethical AI: As AI becomes more integrated with human intelligence,



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], a [Position] at [Company]. I am a [Skill] master in the [Industry] world. I'm excited to meet you and share [Your Interests or Achievements]. Remember, it's a pleasure to meet you! [Your Name]... Continue reading "Hello, my name is [Your Name], a [Position] at [Company]. I am a [Skill] master in the [Industry] world. I'm excited to meet you and share [Your Interests or Achievements]. Remember, it's a pleasure to meet you! " [Your Name]... Continue reading "Hello,

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

Paris is the capital city of France. It is known for its rich history, beautiful architecture, and vibrant cultural scene. The city is home to iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. It also hosts various festivals and events throughout

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 virtual

 healthcare

 assistants

,

 and

In [6]:
llm.shutdown()